In [1]:
map_file_virgo='GW170817_skymap.fits'
map_file='bayestar_no_virgo.fits.gz'
url = 'https://dcc.ligo.org/public/0146/G1701985/001/bayestar_no_virgo.fits.gz'
#url_virgo ='https://dcc.ligo.org/public/0146/G1701985/001/bayestar.fits.gz'
url_virgo ='https://dcc.ligo.org/public/0157/P1800381/007/GW170817_skymap.fits.gz'


In [2]:
%matplotlib notebook

#map_healpix = hp.read_map(map_file)
nside=1024
nested=True

In [8]:
import ligo.skymap.io
import numpy as np
import healpy as hp
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u
import ligo.skymap.plot
from matplotlib import pyplot as plt

In [4]:
#Local
# (map_array, map_header)=ligo.skymap.io.read_sky_map(map_file)
# (map_array_virgo, map_header_virgo)=ligo.skymap.io.read_sky_map(map_file_virgo)

#WEB
(map_array, map_header)=ligo.skymap.io.read_sky_map(url)
(map_array_virgo, map_header_virgo)=ligo.skymap.io.read_sky_map(url_virgo)

map_header_virgo

OrderedDict([('HISTORY',
              ['',
               'Generated by running the following script:',
               'ligo-skymap-from-samples --seed 170817 --outdir . --objid GW170817 --fit',
               'soutname MCMC_TF2_LowSpin_AllSky.fits.gz MCMC_TF2_LowSpin_AllSky_post.da',
               't']),
             ('nest', True),
             ('objid', 'GW170817'),
             ('gps_time', 1187008882.429464),
             ('gps_creation_time', 1241387534.61385),
             ('creator', 'ligo-skymap-from-samples'),
             ('origin', 'LIGO/Virgo'),
             ('distmean', 38.03408225450563),
             ('diststd', 7.499686641911211),
             ('vcs_version', 'ligo.skymap 0.0.15'),
             ('vcs_revision', '65fc6500a1e117fec2e27550a8e9b10c9792ffca'),
             ('build_date', '2018-09-04T14:19:20')])

In [ ]:
#Gets continement probability
indices = np.argsort(-map_array)
region = np.empty(map_array.shape)
region[indices] = 100 * np.cumsum(map_array[indices])

indices_v = np.argsort(-map_array_virgo)
region_v = np.empty(map_array_virgo.shape)
region_v[indices_v] = 100 * np.cumsum(map_array_virgo[indices_v])

center = SkyCoord.from_name('NGC 4993')
center_str = '%fd %fd'%(center.ra.deg, center.dec.deg)

In [6]:
def getfov_img(width=117.0, height=77.0, ra=center.ra.deg, dec=center.dec.deg ):
    
    ra_vertexes= np.array([ra-width/2, ra+width/2,ra+width/2,ra-width/2])
    dec_vertexes=np.array([dec-height/2, dec-height/2,dec+height/2,dec+height/2])

    theta_vert= np.deg2rad(90.-dec_vertexes)
    phi_vert = np.deg2rad(ra_vertexes)

    pix_ver = hp.ang2pix(nside, theta_vert, phi_vert)
    pix_ver_vec=hp.ang2vec(theta_vert, phi_vert)
    fov=hp.query_polygon(nside, pix_ver_vec[0:4], inclusive=True)
    return fov

NameError: name 'center' is not defined

In [ ]:


fig = plt.figure(figsize=(4, 4), dpi=600)
ax = plt.axes(
    [0.05, 0.05, 0.9, 0.9],
    projection='astro globe',
    center=center_str)
# ax_inset = plt.axes(
#     [0.59, 0.3, 0.4, 0.4],
#     projection='astro zoom',
#     center=center_str,
#     radius=10*u.deg)
# for key in ['ra', 'dec']:
#     ax_inset.coords[key].set_ticklabel_visible(False)
#     ax_inset.coords[key].set_ticks_visible(False)
ax.grid()
#ax.mark_inset_axes(ax_inset)
#ax.connect_inset_axes(ax_inset, 'upper left')
#ax.connect_inset_axes(ax_inset, 'lower left')
#ax_inset.scalebar((0.1, 0.1), 5 * u.deg).label()
#ax_inset.compass(0.9, 0.1, 0.2)
#ax.imshow_hpx(map_file, cmap='Blues')
#ax.contourf_hpx(map_file, cmap='Blues')

#ax.contour_hpx(map_array, cmap='cylon', levels=levels)
levels=[50, 90]
ax.contourf_hpx(region, cmap='cylon', levels=levels)

ax.contour_hpx(region_v, cmap='Greens', levels=levels)

fov_region = np.empty(map_array.shape)
fov = getfov_img(width=117.0, height=77.0 )
fov_region[fov]=1.
ax.imshow_hpx(fov_region, cmap='Blues', alpha=0.5)

fov_region = np.empty(map_array.shape)
fov = getfov_img(width=61.0, height=31.0 )
fov_region[fov]=1.
ax.imshow_hpx(fov_region, cmap='Reds', alpha=0.5)


#ax_inset.imshow_hpx(url, cmap='cylon')
# ax_inset.plot(
#     center.ra.deg, center.dec.deg,
#     transform=ax_inset.get_transform('world'),
#     marker=ligo.skymap.plot.reticle(),
#     markersize=30,
#     markeredgewidth=3)

In [ ]:
plt.savefig('GW170817_THESEUS_FOV.tiff')

In [ ]:
hp.mollview(map_array, rot=(ra,dec,0))
hp.graticule()
hp.projplot(theta_vert, phi_vert, 'wx-')
#hp.projplot([0,np.pi/2], [0,np.pi/10], 'wx', linestyle='-')